In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='latin1')
df_test = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin1')
df_sample = pd.read_csv('/kaggle/input/sample100data/sample_tweets.csv',encoding='latin1')

In [ ]:
df_sample2 = df_sample[df_sample['Zhengyang Xu'] != ''][['text','Zhengyang Xu']].reset_index()
df_sample2.columns = ['index', 'text', 'target']
df_sample2 = df_sample2[['text','target']]
df_sample2 = df_sample2.dropna()

In [ ]:
def trans_sentiment(x):
    if x == 'Extremely Positive':
        return 'Positive'
    elif x == 'Extremely Negative':
        return 'Negative'
    else:
        return x

In [ ]:
df_train['Sentiment'] = df_train['Sentiment'].apply(lambda x: trans_sentiment(x) )

In [ ]:
df_train['Sentiment'].value_counts()

In [ ]:
def number_encode(x):
    if x == 'Positive':
        return 2
    elif x == 'Neutral':
        return 1
    else:
        return 0

In [ ]:
df_train['Sentiment_'] = df_train['Sentiment'].apply(lambda x: number_encode(x))

In [ ]:
df_train.OriginalTweet[123]

In [ ]:
import re
def text_clean(x):
    temp = x.lower()
    # links remove
    temp = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',temp)
    # usernames remove
    temp = re.sub('@[^\s]+','', temp)
    # replace hashtags by just words
    temp = re.sub(r'#([^\s]+)', r'\1', temp)
    # correct all multiple white spaces to a single white space
    temp = re.sub('[\s]+', ' ', temp)
    # removing words less than 3 chars
    temp = re.sub(r'\W*\b\w{1,3}\b', '', temp)
    temp = temp.strip()
    return temp

In [ ]:
df_train['Tweets_clean'] = df_train['OriginalTweet'].apply(lambda x: text_clean(x))

In [ ]:
df_sample2['text'] = df_sample2['text'].apply(lambda x: text_clean(x))

In [ ]:
df_sample2['text'][39]

In [ ]:
df_sample2.to_csv('sample_data.csv')

In [ ]:
df_train[['Tweets_clean','Sentiment_']]

In [ ]:
df_2 = df_train[df_train['Tweets_clean'] != ''][['Tweets_clean','Sentiment_']].reset_index()

In [ ]:
df_2.columns = ['index', 'Tweets_clean', 'target']

In [ ]:
df_2.to_csv('sentiment_clean.csv',index=False)

In [ ]:
df_2.head()

In [ ]:
import fasttext

In [ ]:
model = fasttext.train_supervised(input="cooking.train")

In [ ]:
import csv
train = open('tweet3.train','w')
with open('/kaggle/input/clean-sentiment/sentiment_clean.csv', encoding = 'latin1') as csv_file:
    csv_reader = csv.DictReader(csv_file, fieldnames=['index','Tweets_clean','Sentiment_'])
    for row in csv_reader:
        text = row['Tweets_clean']
        print(f'__label__{row["Sentiment_"]} {text}', file=train)

In [ ]:
train

Spelling Correction

## GloVe and RNN

In [ ]:
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['Tweets_clean']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus = create_corpus(df_2)

In [ ]:
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
  
    return maxList, maxLength

In [ ]:
FindMaxLength(corpus)

In [ ]:
embedding_dict={}
with open('/kaggle/input/glove6b/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_LEN=33
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='categorical_crossentropy',optimizer=optimzer,metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
train=tweet_pad[:df_2.shape[0]]
test=tweet_pad[df_2.shape[0]:]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,df_2['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

In [ ]:
from keras.utils import to_categorical
 
one_hot_train_labels = to_categorical(y_train) 
one_hot_test_labels = to_categorical(y_test)


In [ ]:
history=model.fit(X_train,one_hot_train_labels,batch_size=4,epochs=15,validation_data=(X_test,one_hot_test_labels),verbose=2)

In [ ]:
import tensorflow as tf

## traditional machine learning

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
df_3 = df_2
df_3 = df_3[['Tweets_clean','target']]
df_3.columns = ['text','target']

In [ ]:
df_full = [df_3,df_sample2]
df_full = pd.concat(df_full)

In [ ]:
df_full['text'] = df_full['text'].apply(lambda x: text_clean(x))

In [ ]:
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(df_full['text'].values)

In [ ]:
X2 = feature_extraction.fit_transform(df_sample2["text"].values)

In [ ]:
df_full = df_full.reset_index(drop=True)

In [ ]:
X_train = X[:-95]
y_train = df_full['target'][:-95]
X_test = X[-95:]
y_test = df_full['target'][-95:]

In [ ]:
# split into training- and test set
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(X,df_2['target'].values,test_size=0.15)
# print('Shape of train',X_train.shape)
# print("Shape of Validation ",X_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
# from sklearn.metrics import roc_auc_score
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred_2 = model.predict(X_test)


In [ ]:
print("Accuracy xgboost:",metrics.accuracy_score(y_test, y_pred_2))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred_2, digits=3))

## BERT

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df_2['Tweets_clean'],df_2['target'].values,test_size=0.15)

In [ ]:
from bert_sklearn import BertClassifier
model = BertClassifier()         # text/text pair classification
print(model)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.save('covid-bert.model')

In [ ]:
y_pred = model.predict(X_test)          

## prediction

In [ ]:
df_sample2['text'][0]

In [ ]:
from bert_sklearn import BertClassifier
from bert_sklearn import load_model
new_model = load_model('/kaggle/input/covidbert/covid-bert.model')

In [ ]:
new_model.score(df_sample2['text'],df_sample2['target'])

In [ ]:
y_pred_bert = new_model.predict(df_sample2['text'])

In [ ]:
from sklearn import metrics
print(metrics.classification_report(df_sample2['target'], y_pred_bert, digits=3))

In [ ]:
model = BertClassifier()